In [12]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression as lr
import statsmodels.api as sm


### Homework 1.1 Q1 - Q3

In [28]:
# 1. Import Libraries and Load Data
import pandas as pd
import statsmodels.api as sm

# Load the CSV file into the 'df_1_1' DataFrame.
# Ensure the file is in the same directory as your notebook.
try:
    df_1_1 = pd.read_csv('homework_1.1.csv')
    print("File loaded successfully. Data preview:")
    print(df_1_1.head())
except FileNotFoundError:
    print("Error: 'homework_1.1.csv' file not found.")
    df_1_1 = None

File loaded successfully. Data preview:
   Unnamed: 0        X1        X2        X3          Y
0           0 -0.440646 -0.390227  0.156718  -0.877671
1           1 -3.810099 -1.304665 -1.105117 -10.130388
2           2 -1.425451 -0.340049  1.115908   0.284068
3           3 -1.325750  0.161906 -0.254670  -1.994344
4           4  3.120263  1.487343 -1.164839   2.030030


In [30]:
# 2. Perform Multiple Linear Regression
if df_1_1 is not None:
    # Define the dependent (Y) and independent (X1, X2, X3) variables.
    Y = df_1_1['Y']
    X_all = df_1_1[['X1', 'X2', 'X3']]
    
    # Add a constant (intercept) to the model.
    X_all = sm.add_constant(X_all)

    # Fit the Ordinary Least Squares (OLS) model.
    model_all = sm.OLS(Y, X_all).fit()

    # Print the full regression summary.
    print("### Multiple Regression Results (Y ~ X1 + X2 + X3) ###")
    print(model_all.summary())

### Multiple Regression Results (Y ~ X1 + X2 + X3) ###
                            OLS Regression Results                            
Dep. Variable:                      Y   R-squared:                       0.991
Model:                            OLS   Adj. R-squared:                  0.991
Method:                 Least Squares   F-statistic:                 3.543e+04
Date:                Sun, 31 Aug 2025   Prob (F-statistic):               0.00
Time:                        09:13:08   Log-Likelihood:                -727.62
No. Observations:                1000   AIC:                             1463.
Df Residuals:                     996   BIC:                             1483.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------

In [33]:
# 3. Answer Questions 1 & 3
if 'model_all' in locals():
    # --- Question 1: Coefficient of X1 ---
    coeff_x1_full = model_all.params['X1']
    print("\n--- Question 1: Coefficient of X1 ---")
    print(f"The coefficient for X1 in the multiple regression is {coeff_x1_full:.4f}.")

    # --- Question 3: Most Significant Coefficient ---
    t_stats = model_all.tvalues.drop('const')
    most_significant_var = t_stats.abs().idxmax()
    print("--- Question 3: Most Significant Coefficient ---")
    print(f"T-statistics for each variable:\n{t_stats}\n")
    print(f"The variable with the highest absolute t-statistic is '{most_significant_var}'.")


--- Question 1: Coefficient of X1 ---
The coefficient for X1 in the multiple regression is 1.0071.
--- Question 3: Most Significant Coefficient ---
T-statistics for each variable:
X1     60.984011
X2     53.283212
X3    196.645240
dtype: float64

The variable with the highest absolute t-statistic is 'X3'.


In [34]:
# 4. Answer Question 2 (Simple Regression Comparison)
if 'model_all' in locals():
    print("\n" + "="*80 + "\n")
    print("--- Question 2: Greatest Difference in Coefficients ---")
    
    diffs = {}
    
    # Loop through each variable to perform a simple regression.
    for var in ['X1', 'X2', 'X3']:
        # Prepare the data for the simple regression.
        X_simple = sm.add_constant(df_1_1[[var]])
        
        # Fit the simple OLS model.
        model_simple = sm.OLS(Y, X_simple).fit()
        coeff_simple = model_simple.params[var]

        # Get the corresponding coefficient from the multiple regression model.
        coeff_multiple = model_all.params[var]

        # Calculate the absolute difference between the two coefficients.
        diffs[var] = abs(coeff_multiple - coeff_simple)
        
        print(f"\nAnalysis for {var}:")
        print(f"  - Simple regression coefficient: {coeff_simple:.4f}")
        print(f"  - Multiple regression coefficient: {coeff_multiple:.4f}")
        print(f"  - Absolute difference: {diffs[var]:.4f}")

    greatest_diff_var = max(diffs, key=diffs.get)
    print(f"\nThe variable with the greatest difference is '{greatest_diff_var}'.")



--- Question 2: Greatest Difference in Coefficients ---

Analysis for X1:
  - Simple regression coefficient: 1.8418
  - Multiple regression coefficient: 1.0071
  - Absolute difference: 0.8346

Analysis for X2:
  - Simple regression coefficient: 4.0836
  - Multiple regression coefficient: 1.9646
  - Absolute difference: 2.1190

Analysis for X3:
  - Simple regression coefficient: 3.0970
  - Multiple regression coefficient: 2.9755
  - Absolute difference: 0.1216

The variable with the greatest difference is 'X2'.


### Homework 1.2 Q4 - Q7

In [35]:
# 1. Import Libraries and Load Data
import pandas as pd
import numpy as np
from sklearn.neighbors import NearestNeighbors

# Load the CSV file into the 'df_1_2' DataFrame.
try:
    df_1_2 = pd.read_csv('homework_1.2.csv')
    print("File 'homework_1.2.csv' loaded successfully. Data preview:")
    print(df_1_2.head())
except FileNotFoundError:
    print("Error: 'homework_1.2.csv' file not found.")
    df_1_2 = None

File 'homework_1.2.csv' loaded successfully. Data preview:
   Unnamed: 0  X         Y         Z
0           0  0  0.548814  0.548814
1           1  1  1.215189  0.715189
2           2  0  0.602763  0.602763
3           3  0  0.544883  0.544883
4           4  0  0.423655  0.423655


In [36]:
# 2. Prepare Data for Matching
if df_1_2 is not None:
    # Separate the data into treatment and control groups based on the 'X' column.
    treatment = df_1_2[df_1_2['X'] == 1].copy()
    control = df_1_2[df_1_2['X'] == 0].copy()

    # Prepare the 'Z' variable from both groups for matching.
    # Scikit-learn expects a 2D array, so we use [['Z']].
    Z_treatment = treatment[['Z']]
    Z_control = control[['Z']]
    
    print(f"\nData prepared for matching:")
    print(f" - Number of treatment samples (X=1): {len(treatment)}")
    print(f" - Number of control samples (X=0): {len(control)}")


Data prepared for matching:
 - Number of treatment samples (X=1): 48
 - Number of control samples (X=0): 52


In [41]:
# 3. Nearest Neighbor Matching (Questions 4 & 5)
if 'treatment' in locals():
    print("\n" + "="*80 + "\n")
    print("--- Nearest Neighbor Matching (1-to-1) ---")
    
    # Initialize the model to find 1 nearest neighbor.
    nn_model = NearestNeighbors(n_neighbors=1, metric='euclidean')
    
    # Fit the model on the control group's Z values.
    nn_model.fit(Z_control)
    
    # Find the nearest neighbor in the control group for each treatment sample.
    distances, indices = nn_model.kneighbors(Z_treatment)

    # --- Answer for Question 4 ---
    farthest_distance = distances.max()
    print(f"\n--- Question 4: Farthest Match Distance ---")
    print(f"The maximum distance to a nearest neighbor is: {farthest_distance:.4f}")

    
    # --- Answer for Question 5 ---
    # Get the indices of the matched control units.
    matched_control_indices = control.iloc[indices.flatten()].index
    
    # Create the matched control group DataFrame.
    matched_control_group = df_1_2.loc[matched_control_indices]

    # Calculate the average Y for both groups.
    avg_y_treatment = treatment['Y'].mean()
    avg_y_matched_control = matched_control_group['Y'].mean()
    
    # Calculate the treatment effect.
    effect_nn = avg_y_treatment - avg_y_matched_control

    print(f"--- Question 5: Treatment Effect (Nearest Neighbor) ---")
    print(f"Average Y for Treatment Group: {avg_y_treatment:.4f}")
    print(f"Average Y for Matched Control Group: {avg_y_matched_control:.4f}")
    print(f"The estimated effect is: {effect_nn:.4f}")




--- Nearest Neighbor Matching (1-to-1) ---

--- Question 4: Farthest Match Distance ---
The maximum distance to a nearest neighbor is: 0.2102
--- Question 5: Treatment Effect (Nearest Neighbor) ---
Average Y for Treatment Group: 1.1256
Average Y for Matched Control Group: 0.5822
The estimated effect is: 0.5434


In [42]:
# 4. Radius Matching (Questions 6 & 7)
if 'treatment' in locals():
    print("\n" + "="*80 + "\n")
    print("--- Radius Matching (distance < 0.2) ---")
    
    # Initialize the model to find all neighbors within a 0.2 radius.
    radius_model = NearestNeighbors(radius=0.2)
    
    # Fit the model on the control group's Z values.
    radius_model.fit(Z_control)
    
    # Find all neighbors for each treatment sample.
    distances_rad, indices_rad = radius_model.radius_neighbors(Z_treatment)
    
    # --- Answer for Question 6 ---
    # The number of duplicates is the total number of neighbors found
    # minus the number of treatment samples.
    total_neighbors = sum(len(i) for i in indices_rad)
    num_duplicates = total_neighbors - len(treatment)

    print(f"\n--- Question 6: Number of Duplicates ---")
    print(f"Total neighbors found within the radius: {total_neighbors}")
    print(f"The number of duplicates is: {num_duplicates}")
       
    # --- Answer for Question 7 ---
    y_control_means_per_treatment = []
    
    # For each treatment unit, calculate the mean Y of its neighbors.
    for i in range(len(indices_rad)):
        neighbor_df_indices = control.iloc[indices_rad[i]].index
        if len(neighbor_df_indices) > 0:
            mean_y_for_neighbors = df_1_2.loc[neighbor_df_indices]['Y'].mean()
            y_control_means_per_treatment.append(mean_y_for_neighbors)

    # Calculate the average of these means for the overall matched control effect.
    avg_y_radius_matched_control = np.mean(y_control_means_per_treatment)
    
    # The average Y for the treatment group is the same as before.
    avg_y_treatment = treatment['Y'].mean()
    
    # Calculate the treatment effect.
    effect_radius = avg_y_treatment - avg_y_radius_matched_control
    
    print(f"--- Question 7: Treatment Effect (Radius Matching) ---")
    print(f"Average Y for Treatment Group: {avg_y_treatment:.4f}")
    print(f"Average of mean Ys for matched controls: {avg_y_radius_matched_control:.4f}")
    print(f"The estimated effect is: {effect_radius:.4f}")
    



--- Radius Matching (distance < 0.2) ---

--- Question 6: Number of Duplicates ---
Total neighbors found within the radius: 737
The number of duplicates is: 689
--- Question 7: Treatment Effect (Radius Matching) ---
Average Y for Treatment Group: 1.1256
Average of mean Ys for matched controls: 0.5412
The estimated effect is: 0.5844
